## France Public Company Financial Information Crawler
##### Last Update: 2022-08-02
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
EURONEXT MAIN: https://live.euronext.com/en /
EURONEXT Paris: https://live.euronext.com/en/markets/paris/equities/list

Description: This is public company financial statement web crawling code for EURONEXT.com, especially for companies in EURONEXT Paris(France). EURONEXT is an official stock exchange website for several countries in Europe: Netherlands(Amsterdam), Belgium(Brussels), Dublin(Ireland), Portugal(Lisbon), Italy(Milan), Norway(Oslo), and France(Paris). Even though it is an official website, comparably, it does not have a lot of detailed information while it has enough key values, so one should be aware of that. This code can be reused for other countries by editing initial website address and several page button XPATH locations - Netherlands, Belgium, Portugal, Norway, and France is already written and uploaded. In some cases, one might be required to find the way to get full list of a tags on the market while converting. Since the purpose of this code is to collect financial information of public company, this does not collect information of ETFs, Funds, Bonds, and other non-company type entities. If one wants to make crawler for those types of entities, entire code must be newly created for accuracy. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [2]:
import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version
import xlsxwriter

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

In [3]:
path = r'C:\Users\user\Desktop\자료\chromedriver'
print(python_version())
print(pd.__version__)

3.9.12
1.4.2


## 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.
#### - Some directions and explanations are written as comment below.
#### - The order of append must  be depend on the order of column name assigned above.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.

### File Setting
#### - xslx file name must be checked based on user preference.
#### - xlsx file will be newly added on the folder where ipynb file is located.
#### - Column can be changed if it is required, but the main crawler code must be also modified for accuracy.
#### - The way we insert the information on xlsx file is different from csv file.
#### - Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
#### - DO NOT RUN THIS CODE AGAIN AFTER CRAWLING OR THE BLANK FILE WILL REPLACE CURRENT FILE.

### 2021

In [ ]:
# Basic File Setting
workbook = xlsxwriter.Workbook('France_2021.xlsx')
worksheet = workbook.add_worksheet()
row = 0
column = 0
base_url = "https://live.euronext.com"
failList = []
xb = []

# Data Fields(Column Names) that will be collected
content = ['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜']

# Put Column names on the sheet
for item in content :
    worksheet.write(row, column, item)
    column += 1
row += 1

# Get EURONEXT Paris Stock Website
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/paris/equities/list')
time.sleep(1.5)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
time.sleep(3)

# Get each company's links from all pages 1 - 9 page: it should be added/removed by current status
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable1 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable2 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable3 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable4 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable5 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable6 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable7 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable8 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable9 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[6]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable10 = maintable.find_all('a')

# Merge lists
all_atag_maintable = all_atag_maintable1 + all_atag_maintable2 + all_atag_maintable3 + all_atag_maintable4 + all_atag_maintable5 + all_atag_maintable6 + all_atag_maintable7 + all_atag_maintable8 + all_atag_maintable9 + all_atag_maintable10

# Final Link Setting
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
# print(len(company_links))

# Main Data Collection code
for sub in company_links:
    # Start from A column every time
    column = 0
    time.sleep(1)
    # crawling environment setting
    try:
        try:
            driver.get(base_url + sub)
            info = []
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(1.5)
                driver.get(base_url + sub)
                info = []
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                info = []
                time.sleep(3)

        # Counry Information - Must be investigated individually
        info.append("FRA")
        info.append("France")
        info.append("France")
        info.append("UTC+01:00")
        info.append("Europe")
        info.append("3667000000000 USD")
        info.append("67413000")
        info.append("서유럽")

        # Company Name
        name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
        # Unique Code
        info.append("FRA XPAR" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try :
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
            try:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
            except:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
            except:
                try: 
                    info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
                except:
                    info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
            except:
                info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                ("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                info.append("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            except:
                info.append("")
                info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")  
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
        except: 
            # Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            #회계년도
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[2]').text)
            except:
                info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #매출액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "천 유로")
            except:
                info.append("")
            #매출원가금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[2]/td[2]').text + "천 유로")
            except:
                info.append("")
            #판매비관리비금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]').text + "천 유로")
            except:
                info.append("")
            #영업이익손실금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #법인세비용
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[2]').text + "천 유로")
            except:
                info.append("")
            #당기순이익
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[6]/td[2]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        # Financial Information - Default
        except:
            info.append("2020")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        #재무정보중 계산예정            
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
            time.sleep(2)
            try : 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            except :
                try:
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                except:
                    info.append("Public Company")
                    info.append("Public Company")
            try :
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            except :
                try: 
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                except:
                    info.append("Products and Services")
                    info.append("Products and Services")

        except:
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")


        # Language Information
        info.append("FR")
        info.append("Français")
        info.append("Frech")

        # Stock Market Information
        info.append("")
        info.append("EURONEXT Paris")
        info.append("EURONEXT Paris")
        info.append("")
        info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
        time.sleep(2)
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Paris")
        info.append("Paris, Europe")
        info.append("")
        info.append("")

        # Event
        try:
            element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
            href = element.get_attribute('href')
            info.append(href)
        except:
            info.append("")

        # Currency Information
        info.append("EUR")
        info.append("유로")

        # Management
        info.append("Chris")
        info.append("EURONEXT")
        info.append("2022-06-02")
        
        # Status Check
        print("Info Length:")
        print(len(info))  
        
        # Adding Collected Data
        for item in info :
            worksheet.write(row, column, item)
            column += 1
        row += 1
    except:
        try:
            # crawling environment setting
            driver.get(base_url + sub)
            info = []
            time.sleep(2)

            # Counry Information - Must be investigated individually
            info.append("FRA")
            info.append("France")
            info.append("France")
            info.append("UTC+01:00")
            info.append("Europe")
            info.append("3667000000000 USD")
            info.append("67413000")
            info.append("서유럽")

            # Company Code/Name
            name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
            info.append("FRA XPAR" + name)
            info.append(name)
            info.append(name)
 
            #Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            
            # Financial Information - Default
            info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #재무정보중 계산예정            
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            
            # Industry and Products - Default
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")

            # Language Information
            info.append("FR")
            info.append("Français")
            info.append("Frech")

            # Stock Market Information - Default
            info.append("")
            info.append("EURONEXT Paris")
            info.append("EURONEXT Paris")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")

            info.append("")
            info.append("")

            #지점
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("")
            info.append("")

            # Event - Default
            info.append("")

            # Currency Information
            info.append("EUR")
            info.append("유로")

            # Management
            info.append("Chris")
            info.append("EURONEXT")
            info.append("2022-06-02")

            # For Status Checking
            print(info)
            
            #Adding Collected Data
            for item in info :
                worksheet.write(row, column, item)
                column += 1
            row += 1
            failList.append(base_url + sub)
        
        # If not possible, add link to the failList
        except: 
            failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

# Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
workbook.close()

### 2020

In [3]:
# Basic File Setting
workbook = xlsxwriter.Workbook('France_2020.xlsx')
worksheet = workbook.add_worksheet()
row = 0
column = 0
base_url = "https://live.euronext.com"
xb = []

# Data Fields(Column Names) that will be collected
content = ['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜']

# Put Column names on the sheet
for item in content :
    worksheet.write(row, column, item)
    column += 1
row += 1

# Get EURONEXT Paris Stock Website
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/paris/equities/list')
time.sleep(1.5)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
time.sleep(3)

# Get each company's links from all pages 1 - 9 page: it should be added/removed by current status
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable1 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable2 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable3 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable4 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable5 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable6 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable7 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable8 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable9 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[6]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable10 = maintable.find_all('a')

# Merge lists
all_atag_maintable = all_atag_maintable1 + all_atag_maintable2 + all_atag_maintable3 + all_atag_maintable4 + all_atag_maintable5 + all_atag_maintable6 + all_atag_maintable7 + all_atag_maintable8 + all_atag_maintable9 + all_atag_maintable10

# Final Link Setting
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
# print(len(company_links))

# Main Data Collection code
for sub in company_links:
    # Start from A column every time
    column = 0
    time.sleep(1)
    # crawling environment setting
    try:
        try:
            driver.get(base_url + sub)
            info = []
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(1.5)
                driver.get(base_url + sub)
                info = []
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                info = []
                time.sleep(3)

        # Counry Information - Must be investigated individually
        info.append("FRA")
        info.append("France")
        info.append("France")
        info.append("UTC+01:00")
        info.append("Europe")
        info.append("3667000000000 USD")
        info.append("67413000")
        info.append("서유럽")

        # Company Name
        name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
        # Unique Code
        info.append("FRA XPAR" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try :
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
            try:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
            except:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
            except:
                try: 
                    info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
                except:
                    info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
            except:
                info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                ("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                info.append("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            except:
                info.append("")
                info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")  
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
        except: 
            # Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            #회계년도
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[3]').text)
            except:
                info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #매출액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[3]').text + "천 유로")
            except:
                info.append("")
            #매출원가금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[2]/td[3]').text + "천 유로")
            except:
                info.append("")
            #판매비관리비금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[3]').text + "천 유로")
            except:
                info.append("")
            #영업이익손실금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[3]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #법인세비용
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[3]').text + "천 유로")
            except:
                info.append("")
            #당기순이익
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[6]/td[3]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        # Financial Information - Default
        except:
            info.append("2020")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        #재무정보중 계산예정            
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
            time.sleep(2)
            try : 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            except :
                try:
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                except:
                    info.append("Public Company")
                    info.append("Public Company")
            try :
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            except :
                try: 
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                except:
                    info.append("Products and Services")
                    info.append("Products and Services")

        except:
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")


        # Language Information
        info.append("FR")
        info.append("Français")
        info.append("Frech")

        # Stock Market Information
        info.append("")
        info.append("EURONEXT Paris")
        info.append("EURONEXT Paris")
        info.append("")
        info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
        time.sleep(2)
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Paris")
        info.append("Paris, Europe")
        info.append("")
        info.append("")

        # Event
        try:
            element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
            href = element.get_attribute('href')
            info.append(href)
        except:
            info.append("")

        # Currency Information
        info.append("EUR")
        info.append("유로")

        # Management
        info.append("Chris")
        info.append("EURONEXT")
        info.append("2022-06-02")
        
        # Status Check
        print("Info Length:")
        print(len(info))  
        
        # Adding Collected Data
        for item in info :
            worksheet.write(row, column, item)
            column += 1
        row += 1
    except:
        try:
            # crawling environment setting
            driver.get(base_url + sub)
            info = []
            time.sleep(2)

            # Counry Information - Must be investigated individually
            info.append("FRA")
            info.append("France")
            info.append("France")
            info.append("UTC+01:00")
            info.append("Europe")
            info.append("3667000000000 USD")
            info.append("67413000")
            info.append("서유럽")

            # Company Code/Name
            name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
            info.append("FRA XPAR" + name)
            info.append(name)
            info.append(name)
 
            #Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            
            # Financial Information - Default
            info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #재무정보중 계산예정            
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            
            # Industry and Products - Default
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")

            # Language Information
            info.append("FR")
            info.append("Français")
            info.append("Frech")

            # Stock Market Information - Default
            info.append("")
            info.append("EURONEXT Paris")
            info.append("EURONEXT Paris")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")

            info.append("")
            info.append("")

            #지점
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("")
            info.append("")

            # Event - Default
            info.append("")

            # Currency Information
            info.append("EUR")
            info.append("유로")

            # Management
            info.append("Chris")
            info.append("EURONEXT")
            info.append("2022-06-02")

            # For Status Checking
            print(info)
            
            #Adding Collected Data
            for item in info :
                worksheet.write(row, column, item)
                column += 1
            row += 1
            failList.append(base_url + sub)
        
        # If not possible, add link to the failList
        except: 
            failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

# Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
workbook.close()

C:\Users\user\AppData\Local\Temp\ipykernel_20352\2717590243.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
1

Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
1

### 2019

In [4]:
# Basic File Setting
workbook = xlsxwriter.Workbook('France_2019.xlsx')
worksheet = workbook.add_worksheet()
row = 0
column = 0
base_url = "https://live.euronext.com"
xb = []

# Data Fields(Column Names) that will be collected
content = ['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜']

# Put Column names on the sheet
for item in content :
    worksheet.write(row, column, item)
    column += 1
row += 1

# Get EURONEXT Paris Stock Website
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/paris/equities/list')
time.sleep(1.5)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
time.sleep(3)

# Get each company's links from all pages 1 - 9 page: it should be added/removed by current status
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable1 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable2 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable3 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable4 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable5 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable6 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable7 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable8 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable9 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[6]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable10 = maintable.find_all('a')

# Merge lists
all_atag_maintable = all_atag_maintable1 + all_atag_maintable2 + all_atag_maintable3 + all_atag_maintable4 + all_atag_maintable5 + all_atag_maintable6 + all_atag_maintable7 + all_atag_maintable8 + all_atag_maintable9 + all_atag_maintable10

# Final Link Setting
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
# print(len(company_links))

# Main Data Collection code
for sub in company_links:
    # Start from A column every time
    column = 0
    time.sleep(1)
    # crawling environment setting
    try:
        try:
            driver.get(base_url + sub)
            info = []
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(1.5)
                driver.get(base_url + sub)
                info = []
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                info = []
                time.sleep(3)

        # Counry Information - Must be investigated individually
        info.append("FRA")
        info.append("France")
        info.append("France")
        info.append("UTC+01:00")
        info.append("Europe")
        info.append("3667000000000 USD")
        info.append("67413000")
        info.append("서유럽")

        # Company Name
        name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
        # Unique Code
        info.append("FRA XPAR" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try :
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
            try:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
            except:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
            except:
                try: 
                    info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
                except:
                    info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
            except:
                info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                ("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                info.append("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            except:
                info.append("")
                info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")  
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
        except: 
            # Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            #회계년도
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[4]').text)
            except:
                info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #매출액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[4]').text + "천 유로")
            except:
                info.append("")
            #매출원가금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[2]/td[4]').text + "천 유로")
            except:
                info.append("")
            #판매비관리비금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[4]').text + "천 유로")
            except:
                info.append("")
            #영업이익손실금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[4]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #법인세비용
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[4]').text + "천 유로")
            except:
                info.append("")
            #당기순이익
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[6]/td[4]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        # Financial Information - Default
        except:
            info.append("2020")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        #재무정보중 계산예정            
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
            time.sleep(2)
            try : 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            except :
                try:
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                except:
                    info.append("Public Company")
                    info.append("Public Company")
            try :
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            except :
                try: 
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                except:
                    info.append("Products and Services")
                    info.append("Products and Services")

        except:
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")


        # Language Information
        info.append("FR")
        info.append("Français")
        info.append("Frech")

        # Stock Market Information
        info.append("")
        info.append("EURONEXT Paris")
        info.append("EURONEXT Paris")
        info.append("")
        info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
        time.sleep(2)
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Paris")
        info.append("Paris, Europe")
        info.append("")
        info.append("")

        # Event
        try:
            element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
            href = element.get_attribute('href')
            info.append(href)
        except:
            info.append("")

        # Currency Information
        info.append("EUR")
        info.append("유로")

        # Management
        info.append("Chris")
        info.append("EURONEXT")
        info.append("2022-06-02")
        
        # Status Check
        print("Info Length:")
        print(len(info))  
        
        # Adding Collected Data
        for item in info :
            worksheet.write(row, column, item)
            column += 1
        row += 1
    except:
        try:
            # crawling environment setting
            driver.get(base_url + sub)
            info = []
            time.sleep(2)

            # Counry Information - Must be investigated individually
            info.append("FRA")
            info.append("France")
            info.append("France")
            info.append("UTC+01:00")
            info.append("Europe")
            info.append("3667000000000 USD")
            info.append("67413000")
            info.append("서유럽")

            # Company Code/Name
            name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
            info.append("FRA XPAR" + name)
            info.append(name)
            info.append(name)
 
            #Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            
            # Financial Information - Default
            info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #재무정보중 계산예정            
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            
            # Industry and Products - Default
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")

            # Language Information
            info.append("FR")
            info.append("Français")
            info.append("Frech")

            # Stock Market Information - Default
            info.append("")
            info.append("EURONEXT Paris")
            info.append("EURONEXT Paris")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")

            info.append("")
            info.append("")

            #지점
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("")
            info.append("")

            # Event - Default
            info.append("")

            # Currency Information
            info.append("EUR")
            info.append("유로")

            # Management
            info.append("Chris")
            info.append("EURONEXT")
            info.append("2022-06-02")

            # For Status Checking
            print(info)
            
            #Adding Collected Data
            for item in info :
                worksheet.write(row, column, item)
                column += 1
            row += 1
            failList.append(base_url + sub)
        
        # If not possible, add link to the failList
        except: 
            failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

# Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
workbook.close()

C:\Users\user\AppData\Local\Temp\ipykernel_20352\669082260.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
1

Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
1

### 2018

In [5]:
# Basic File Setting
workbook = xlsxwriter.Workbook('France_2018.xlsx')
worksheet = workbook.add_worksheet()
row = 0
column = 0
base_url = "https://live.euronext.com"
xb = []

# Data Fields(Column Names) that will be collected
content = ['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜']

# Put Column names on the sheet
for item in content :
    worksheet.write(row, column, item)
    column += 1
row += 1

# Get EURONEXT Paris Stock Website
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/paris/equities/list')
time.sleep(1.5)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
time.sleep(3)

# Get each company's links from all pages 1 - 9 page: it should be added/removed by current status
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable1 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable2 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable3 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable4 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable5 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable6 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable7 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable8 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable9 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[6]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable10 = maintable.find_all('a')

# Merge lists
all_atag_maintable = all_atag_maintable1 + all_atag_maintable2 + all_atag_maintable3 + all_atag_maintable4 + all_atag_maintable5 + all_atag_maintable6 + all_atag_maintable7 + all_atag_maintable8 + all_atag_maintable9 + all_atag_maintable10

# Final Link Setting
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
# print(len(company_links))

# Main Data Collection code
for sub in company_links:
    # Start from A column every time
    column = 0
    time.sleep(1)
    # crawling environment setting
    try:
        try:
            driver.get(base_url + sub)
            info = []
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(1.5)
                driver.get(base_url + sub)
                info = []
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                info = []
                time.sleep(3)

        # Counry Information - Must be investigated individually
        info.append("FRA")
        info.append("France")
        info.append("France")
        info.append("UTC+01:00")
        info.append("Europe")
        info.append("3667000000000 USD")
        info.append("67413000")
        info.append("서유럽")

        # Company Name
        name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
        # Unique Code
        info.append("FRA XPAR" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try :
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
            try:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
            except:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
            except:
                try: 
                    info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
                except:
                    info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
            except:
                info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                ("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                info.append("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            except:
                info.append("")
                info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")  
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
        except: 
            # Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            #회계년도
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[5]').text)
            except:
                info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #매출액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[5]').text + "천 유로")
            except:
                info.append("")
            #매출원가금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[2]/td[5]').text + "천 유로")
            except:
                info.append("")
            #판매비관리비금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[5]').text + "천 유로")
            except:
                info.append("")
            #영업이익손실금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[5]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #법인세비용
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[5]').text + "천 유로")
            except:
                info.append("")
            #당기순이익
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[6]/td[5]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        # Financial Information - Default
        except:
            info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        #재무정보중 계산예정            
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
            time.sleep(2)
            try : 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            except :
                try:
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                except:
                    info.append("Public Company")
                    info.append("Public Company")
            try :
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            except :
                try: 
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                except:
                    info.append("Products and Services")
                    info.append("Products and Services")

        except:
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")


        # Language Information
        info.append("FR")
        info.append("Français")
        info.append("Frech")

        # Stock Market Information
        info.append("")
        info.append("EURONEXT Paris")
        info.append("EURONEXT Paris")
        info.append("")
        info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
        time.sleep(2)
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Paris")
        info.append("Paris, Europe")
        info.append("")
        info.append("")

        # Event
        try:
            element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
            href = element.get_attribute('href')
            info.append(href)
        except:
            info.append("")

        # Currency Information
        info.append("EUR")
        info.append("유로")

        # Management
        info.append("Chris")
        info.append("EURONEXT")
        info.append("2022-06-02")
        
        # Status Check
        print("Info Length:")
        print(len(info))  
        
        # Adding Collected Data
        for item in info :
            worksheet.write(row, column, item)
            column += 1
        row += 1
    except:
        try:
            # crawling environment setting
            driver.get(base_url + sub)
            info = []
            time.sleep(2)

            # Counry Information - Must be investigated individually
            info.append("FRA")
            info.append("France")
            info.append("France")
            info.append("UTC+01:00")
            info.append("Europe")
            info.append("3667000000000 USD")
            info.append("67413000")
            info.append("서유럽")

            # Company Code/Name
            name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
            info.append("FRA XPAR" + name)
            info.append(name)
            info.append(name)
 
            #Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            
            # Financial Information - Default
            info.append("2021")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #재무정보중 계산예정            
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            
            # Industry and Products - Default
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")

            # Language Information
            info.append("FR")
            info.append("Français")
            info.append("Frech")

            # Stock Market Information - Default
            info.append("")
            info.append("EURONEXT Paris")
            info.append("EURONEXT Paris")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")

            info.append("")
            info.append("")

            #지점
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("")
            info.append("")

            # Event - Default
            info.append("")

            # Currency Information
            info.append("EUR")
            info.append("유로")

            # Management
            info.append("Chris")
            info.append("EURONEXT")
            info.append("2022-06-02")

            # For Status Checking
            print(info) 
            
            #Adding Collected Data
            for item in info :
                worksheet.write(row, column, item)
                column += 1
            row += 1
            failList.append(base_url + sub)
        
        # If not possible, add link to the failList
        except: 
            failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

# Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
workbook.close()

C:\Users\user\AppData\Local\Temp\ipykernel_20352\1703176093.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
1

Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
111
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
1

### 2017

In [5]:
# Basic File Setting
workbook = xlsxwriter.Workbook('France_2017.xlsx')
worksheet = workbook.add_worksheet()
row = 0
column = 0
base_url = "https://live.euronext.com"
xb = []

# Data Fields(Column Names) that will be collected
content = ['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜']

# Put Column names on the sheet
for item in content :
    worksheet.write(row, column, item)
    column += 1
row += 1

# Get EURONEXT Paris Stock Website
driver = webdriver.Chrome(path)
driver.get('https://live.euronext.com/en/markets/paris/equities/list')
time.sleep(1.5)
driver.find_element(By.XPATH, value='/html/body/div[3]/div/div/div[3]/button[1]').click()
time.sleep(3)

# Get each company's links from all pages 1 - 9 page: it should be added/removed by current status
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable1 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable2 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable3 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable4 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable5 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable6 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable7 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable8 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/a[2]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable9 = maintable.find_all('a')

driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div[2]/div/main/section/div[3]/div/div/div[1]/div[2]/div[4]/div/span/a[6]').click()
time.sleep(3)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('table', id='stocks-data-table')
all_atag_maintable10 = maintable.find_all('a')

# Merge lists
all_atag_maintable = all_atag_maintable1 + all_atag_maintable2 + all_atag_maintable3 + all_atag_maintable4 + all_atag_maintable5 + all_atag_maintable6 + all_atag_maintable7 + all_atag_maintable8 + all_atag_maintable9 + all_atag_maintable10

# Final Link Setting
company_links = []
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    

# To check if all companies are selected
# print(len(company_links))

# Main Data Collection code
for sub in company_links:
    # Start from A column every time
    column = 0
    time.sleep(1)
    # crawling environment setting
    try:
        try:
            driver.get(base_url + sub)
            info = []
            time.sleep(4)
        except:
            try:
                driver.get("https://www.google.com/")
                time.sleep(1.5)
                driver.get(base_url + sub)
                info = []
                time.sleep(3)
            except:
                driver.get(base_url + sub)
                info = []
                time.sleep(3)

        # Counry Information - Must be investigated individually
        info.append("FRA")
        info.append("France")
        info.append("France")
        info.append("UTC+01:00")
        info.append("Europe")
        info.append("3667000000000 USD")
        info.append("67413000")
        info.append("서유럽")

        # Company Name
        name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
        # Unique Code
        info.append("FRA XPAR" + name)
        info.append(name)
        info.append(name)

        #Descripton, Contact, Address, Extra Information, Management Information
        try :
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            address = driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[1]/div[2]').text
            try:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Address: " + address)
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT AMSTERDAM(Netherlands). Their net sales were " + driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]').text + "K Euros, mainly from selling products(services). Company Address: " + address + "You can find more information on company's website. ")
            except:
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
                info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[3]/div/div/div/div[2]/div[2]/a').text) 
            except:
                try: 
                    info.append(driver.find_elemen(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
                except:
                    info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[3]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[3]/div/a').text)
            except:
                info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                ("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(2)').text)
            except:
                info.append("Paris")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            try:
                info.append(driver.find_element(By.CSS_SELECTOR, value='#home-office-address-column1-wrapper > div:nth-child(1) > div:nth-child(3)').text)
            except:
                info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[1]').text)
            except:
                info.append("")
                info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[1]/div/div/div/div/div/div[1]/div[1]/h1/strong').text)
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[1]/div/div/div/div[2]/address/div[2]/div[2]/a').text)
            except:
                info.append("")
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text)
            except:
                info.append("")  
            try :
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/section/div[2]/div/div/div/div[2]/div[2]/a').text) 
            except:
                info.append("")
        except: 
            # Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")

        # Financial Information
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[3]').click()
            time.sleep(2)
            #회계년도
            try: 
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/thead/tr/th[6]').text)
            except:
                info.append("")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #매출액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[6]').text + "천 유로")
            except:
                info.append("")
            #매출원가금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[2]/td[6]').text + "천 유로")
            except:
                info.append("")
            #판매비관리비금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[6]').text + "천 유로")
            except:
                info.append("")
            #영업이익손실금액
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[6]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #법인세비용
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[6]').text + "천 유로")
            except:
                info.append("")
            #당기순이익
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div/div/div/div/div/div[2]/div[3]/div[2]/table/tbody/tr[6]/td[6]').text + "천 유로")
            except:
                info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        # Financial Information - Default
        except:
            info.append("2020")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
        #재무정보중 계산예정            
        info.append("")
        info.append("")
        info.append("")
        info.append("")
        info.append("")

        # Industry and Products
        try:
            driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[2]').click()
            time.sleep(2)
            try : 
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(2) > td:nth-child(2) > strong').text)
            except :
                try:
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(1) > td:nth-child(2) > strong').text)
                except:
                    info.append("Public Company")
                    info.append("Public Company")
            try :
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
                info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(4) > td:nth-child(2) > strong').text)
            except :
                try: 
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                    info.append(driver.find_element(By.CSS_SELECTOR, value='#fs_icb_block > div > div.card-body > div.table-responsive > table > tbody > tr:nth-child(3) > td:nth-child(2) > strong').text)
                except:
                    info.append("Products and Services")
                    info.append("Products and Services")

        except:
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")


        # Language Information
        info.append("FR")
        info.append("Français")
        info.append("Frech")

        # Stock Market Information
        info.append("")
        info.append("EURONEXT Paris")
        info.append("EURONEXT Paris")
        info.append("")
        info.append("")
        driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/section/div[2]/div/div/div/div/nav/div/a[1]').click()
        time.sleep(2)
        try:
            info.append(driver.find_element(By.CSS_SELECTOR, value='#header-instrument-price').text + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[2]').text  + " EUR")
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[3]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/section/div[2]/div/div/div[1]/div[2]/div/div[1]/div/table/tbody/tr[1]/td[5]').text  + " EUR")
        except:
            info.append("")
        info.append("")
        info.append("")
        info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[3]/td[2]').text)
        except:
            info.append("")
        try:
            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div[1]/div/div/div[1]/div/div[1]/main/section/div[2]/div/div/div[1]/div/div/div/div[1]/div/table/tbody/tr[12]/td[2]').text)
        except:
            info.append("")

        #지점
        info.append("")
        info.append("")
        info.append("Paris")
        info.append("Paris, Europe")
        info.append("")
        info.append("")

        # Event
        try:
            element = driver.find_element(By.CSS_SELECTOR, value='#content-left > section > div.container-full > div.card.mb-4.text-left > div.card-body > div > div > div > table > tbody > tr:nth-child(1) > td:nth-child(2) > a')
            href = element.get_attribute('href')
            info.append(href)
        except:
            info.append("")

        # Currency Information
        info.append("EUR")
        info.append("유로")

        # Management
        info.append("Chris")
        info.append("EURONEXT")
        info.append("2022-06-02")
        
        # Status Check
        print("Info Length:")
        print(len(info))  
        
        # Adding Collected Data
        for item in info :
            worksheet.write(row, column, item)
            column += 1
        row += 1
    except:
        try:
            # crawling environment setting
            driver.get(base_url + sub)
            info = []
            time.sleep(2)

            # Counry Information - Must be investigated individually
            info.append("FRA")
            info.append("France")
            info.append("France")
            info.append("UTC+01:00")
            info.append("Europe")
            info.append("3667000000000 USD")
            info.append("67413000")
            info.append("서유럽")

            # Company Code/Name
            name = driver.find_element(By.CSS_SELECTOR, value='#header-instrument-name > strong').text
            info.append("FRA XPAR" + name)
            info.append(name)
            info.append(name)
 
            #Descripton, Contact, Address, Extra Information, Management Information - Default
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France).")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append(name + "(English: " + name + ")" + " is a public company that is listed on EURONEXT PARIS(France). You can find more information on company's website.")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("Paris, Europe")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("Active") 
            info.append("")
            info.append("")
            info.append("Contact")
            info.append("Contact")
            info.append("")
            info.append("")
            info.append("")
            info.append(str(base_url + sub))
            info.append("")
            
            # Financial Information - Default
            info.append("2021")  
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("") 
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            #재무정보중 계산예정            
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            
            # Industry and Products - Default
            info.append("Public Company")
            info.append("Public Company")
            info.append("Products and Services")
            info.append("Products and Services")

            # Language Information
            info.append("FR")
            info.append("Français")
            info.append("Frech")

            # Stock Market Information - Default
            info.append("")
            info.append("EURONEXT Paris")
            info.append("EURONEXT Paris")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")
            info.append("")

            info.append("")
            info.append("")

            #지점
            info.append("")
            info.append("")
            info.append("Paris")
            info.append("Paris, Europe")
            info.append("")
            info.append("")

            # Event - Default
            info.append("")

            # Currency Information
            info.append("EUR")
            info.append("유로")

            # Management
            info.append("Chris")
            info.append("EURONEXT")
            info.append("2022-06-02")

            # For Status Checking
            print(info)
            
            #Adding Collected Data
            for item in info :
                worksheet.write(row, column, item)
                column += 1
            row += 1
            failList.append(base_url + sub)
        
        # If not possible, add link to the failList
        except: 
            failList.append(base_url + sub)
            
# Check failed result 
# - There are many ways to deal with failList: retry, ignore, manually add, find why error has occured.
# - This will be up to user's choice.
print("failLength:")
print(len(failList))
print(failList)

# Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
workbook.close()

C:\Users\user\AppData\Local\Temp\ipykernel_19040\3553739850.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
1

Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
1

Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
111
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
110
Info Length:
109
Info Length:
109
Info Length:
109
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
110
Info Length:
109
Info Length:
109
failLength:
9
['https://live.euronext.com/en/product/equities/FR0012616852-XPAR/abionyx-pharma/abnx/quotes', 'https://live.euronext.com/en/product/equities/FR0013258662-XPAR/ald/ald/quotes', 'https://live.euronext.com/en/product/equities/FR0004023208-XPAR/bassac/bass/quotes', 'https://live.euronext.com/en/product/equities/FR0010844001-ALXP/eurasia-groupe/aleua/quotes', 'https://live.euronext.com/en/product/equities/FR0012419307-ALXP/focusentertainment/alfoc/quotes', 'https://live.euronext.com/en/product/equities/FR0000065930-XPAR/fonciere-7-invest/lebl/quotes', 'https://live.euronext.com/en/product/equities/FR000003849

## 5. FailList
#### - There are many ways to deal with elements failList: retry(if caused by server loading status), ignore(if caused by actually non-existing data), manually add(if only few exist), find why error has occurred(if massive).

In [ ]:
print(len(failList))
print(failList)